<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_D1_MongoDB_Local_CRUD_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

Based on https://github.com/giffy/MongoDB_PyMongo_Tutorial

#Installs 

In [1]:
from pymongo import MongoClient
import pandas as pd
import dateutil

In [2]:
!apt install mongodb > /dev/null
 
#Local Server Need Not be started for Remote Operations
!service mongodb start



 * Starting database mongodb
   ...done.


In [3]:
mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']

['admin', 'local']

# Create Database objects

In [4]:
# Database PraxisDB will be created only when we create collections, or documents in it
mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() 

['admin', 'local']

In [12]:
# Delete a database
client.drop_database('PraxisDB')

In [13]:
# Create a new collection. We can create the collection or leave to MongoDB to create it as soon as a document is generated.
db = client.PraxisDB
db.create_collection('addresses')       # Optional collection creation 

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PraxisDB'), 'addresses')

In [14]:
# Show Collections. Query returns a Cursor [ ] ,  list it to see the content
list (db.list_collections())
# empty list '[]' means that there are not collections in database

[{'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'PraxisDB.addresses',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': UUID('976d2133-a8a4-41c8-8998-320b7d096407')},
  'name': 'addresses',
  'options': {},
  'type': 'collection'}]

In [15]:
# Set the collection to work with
kollection = db.addresses
kollection.insert_one({'name' : 'jordi'})     # Insert one item to create the collection
list (kollection.find())                  # Show the existing collections

[{'_id': ObjectId('619dacd1c365f4331ba5e26c'), 'name': 'jordi'}]

In [16]:
# Rename a collections
db.addresses.rename('addressbook')
kollection2 = db.addressbook                   # Set the collection to work with
list (db.list_collections())                  # Show the existing collections

[{'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'PraxisDB.addressbook',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': UUID('976d2133-a8a4-41c8-8998-320b7d096407')},
  'name': 'addressbook',
  'options': {},
  'type': 'collection'}]

In [17]:
# Delete collection
db.drop_collection('addressbook')

{'nIndexesWas': 1, 'ns': 'PraxisDB.addressbook', 'ok': 1.0}

In [18]:
client.list_database_names() 

['admin', 'config', 'local']

# Operations

## Single Row Insert

In [41]:
cDoc1 = {  'name' : 'Prithwis' ,                                     # String 
          'age' : '59',                                             # Integer
          'gender' : 'Male',                                        # String 
          'likes_code' : True,                                      # Boolean
          'address': {
              'street' : 'Panditiya Road',                          # String ( special character with escape \ )
              'number' : 36,                                        # Integer
              'city' : 'Kolkata',                                   # String 
              'floor' : None,                                       # Null 
              'postalcode' : '700029',                              # String containing a number
              },
          'favouritefruits': ['apple','pineapple','orange']         # Array        
       }

In [42]:
# Define database, collection etc
db = client.PraxisDB
# Delete collection
db.drop_collection('AddressBook')
db.create_collection('AddressBook')       # Optional collection creation 
kollection3 = db.AddressBook                   # Set the collection to work with
insert_result = kollection3.insert_one( cDoc1)
insert_result.acknowledged    # Confirms that insert is successful

True

In [43]:
insert_result.inserted_id     # Shows the document ID 

ObjectId('619db1c0c365f4331ba5e274')

In [44]:
db.AddressBook.find()  # returns a cursor

In [45]:
#c = db.AddressBook.find()
c = kollection3.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('619db1c0c365f4331ba5e274'), 'name': 'Prithwis', 'age': '59', 'gender': 'Male', 'likes_code': True, 'address': {'street': 'Panditiya Road', 'number': 36, 'city': 'Kolkata', 'floor': None, 'postalcode': '700029'}, 'favouritefruits': ['apple', 'pineapple', 'orange']}


In [46]:
cDoc2 = {  'name' : 'Charanpreet' ,                                     # String 
          'age' : '57',                                             # Integer
          'gender' : 'Male',                                        # String 
          'likes_code' : False,                                      # Boolean
          'address': {
              'street' : None,                          # String ( special character with escape \ )
              'number' : None,                                        # Integer
              'city' : 'Kolkata',                                   # String 
              'floor' : None,                                       # Null 
              'postalcode' : '700001',                              # String containing a number
              },
          'favouritefruits': ['apple','guava','mango']         # Array        
       }

In [47]:
insert_result = kollection3.insert_one( cDoc2)
insert_result.acknowledged    # Confirms that insert is successful

True

In [48]:
#c = db.AddressBook.find()
c = kollection3.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('619db1c0c365f4331ba5e274'), 'name': 'Prithwis', 'age': '59', 'gender': 'Male', 'likes_code': True, 'address': {'street': 'Panditiya Road', 'number': 36, 'city': 'Kolkata', 'floor': None, 'postalcode': '700029'}, 'favouritefruits': ['apple', 'pineapple', 'orange']}
{'_id': ObjectId('619db1e3c365f4331ba5e275'), 'name': 'Charanpreet', 'age': '57', 'gender': 'Male', 'likes_code': False, 'address': {'street': None, 'number': None, 'city': 'Kolkata', 'floor': None, 'postalcode': '700001'}, 'favouritefruits': ['apple', 'guava', 'mango']}


In [49]:
cDoc3 = {  'name' : 'Narendra' ,                                     # String 
          'age' : '67',                                             # Integer
          'gender' : 'Male',                                        # String 
          'likes_code' : False,                                      # Boolean
          'address': {
              'street' : None,                          # String ( special character with escape \ )
              'number' : None,                                        # Integer
              'city' : 'Delhi',                                   # String 
              'floor' : None,                                       # Null 
              'postalcode' : '700001',                              # String containing a number
              },
          'favouritefruits': ['apple','guava','mango']         # Array        
       }

cDoc4 = {  'name' : 'Aditya' ,                                     # String 
          'age' : '47',                                             # Integer
          'gender' : 'Male',                                        # String 
          'likes_code' : False,                                      # Boolean
          'address': {
              'street' : None,                          # String ( special character with escape \ )
              'number' : None,                                        # Integer
              'city' : 'Lucknow',                                   # String 
              'floor' : None,                                       # Null 
              'postalcode' : '700001',                              # String containing a number
              },
          'favouritefruits': ['apple','guava','mango']         # Array        
       }

In [52]:
insert_result = kollection3.insert_many([cDoc3,cDoc4])

In [53]:
#c = db.AddressBook.find()
c = kollection3.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('619db1c0c365f4331ba5e274'), 'name': 'Prithwis', 'age': '59', 'gender': 'Male', 'likes_code': True, 'address': {'street': 'Panditiya Road', 'number': 36, 'city': 'Kolkata', 'floor': None, 'postalcode': '700029'}, 'favouritefruits': ['apple', 'pineapple', 'orange']}
{'_id': ObjectId('619db1e3c365f4331ba5e275'), 'name': 'Charanpreet', 'age': '57', 'gender': 'Male', 'likes_code': False, 'address': {'street': None, 'number': None, 'city': 'Kolkata', 'floor': None, 'postalcode': '700001'}, 'favouritefruits': ['apple', 'guava', 'mango']}
{'_id': ObjectId('619db16bc365f4331ba5e272'), 'name': 'Charanpreet21', 'age': '57', 'gender': 'Male', 'likes_code': False, 'address': {'street': None, 'number': None, 'city': 'Kolkata', 'floor': None, 'postalcode': '700001'}, 'favouritefruits': ['apple', 'guava', 'mango']}
{'_id': ObjectId('619db16bc365f4331ba5e273'), 'name': 'Charanpreet22', 'age': '57', 'gender': 'Male', 'likes_code': False, 'address': {'street': None, 'number': None, 'cit

## Read

In [54]:
list(kollection3.find())

[{'_id': ObjectId('619db1c0c365f4331ba5e274'),
  'address': {'city': 'Kolkata',
   'floor': None,
   'number': 36,
   'postalcode': '700029',
   'street': 'Panditiya Road'},
  'age': '59',
  'favouritefruits': ['apple', 'pineapple', 'orange'],
  'gender': 'Male',
  'likes_code': True,
  'name': 'Prithwis'},
 {'_id': ObjectId('619db1e3c365f4331ba5e275'),
  'address': {'city': 'Kolkata',
   'floor': None,
   'number': None,
   'postalcode': '700001',
   'street': None},
  'age': '57',
  'favouritefruits': ['apple', 'guava', 'mango'],
  'gender': 'Male',
  'likes_code': False,
  'name': 'Charanpreet'},
 {'_id': ObjectId('619db16bc365f4331ba5e272'),
  'address': {'city': 'Kolkata',
   'floor': None,
   'number': None,
   'postalcode': '700001',
   'street': None},
  'age': '57',
  'favouritefruits': ['apple', 'guava', 'mango'],
  'gender': 'Male',
  'likes_code': False,
  'name': 'Charanpreet21'},
 {'_id': ObjectId('619db16bc365f4331ba5e273'),
  'address': {'city': 'Kolkata',
   'floor': N

In [55]:
#list ( collection.find( {'name' : 'Charanpreet' } ))   
list ( kollection3.find( {'name' : 'Prithwis' } ))   

[{'_id': ObjectId('619db1c0c365f4331ba5e274'),
  'address': {'city': 'Kolkata',
   'floor': None,
   'number': 36,
   'postalcode': '700029',
   'street': 'Panditiya Road'},
  'age': '59',
  'favouritefruits': ['apple', 'pineapple', 'orange'],
  'gender': 'Male',
  'likes_code': True,
  'name': 'Prithwis'}]

In [56]:
list ( kollection3.find( {'address.postalcode' : '700029' } ))             # find, can use one key or more 

[{'_id': ObjectId('619db1c0c365f4331ba5e274'),
  'address': {'city': 'Kolkata',
   'floor': None,
   'number': 36,
   'postalcode': '700029',
   'street': 'Panditiya Road'},
  'age': '59',
  'favouritefruits': ['apple', 'pineapple', 'orange'],
  'gender': 'Male',
  'likes_code': True,
  'name': 'Prithwis'}]

## Update

In [57]:
## Update an existing document
update_result = kollection3.update_one( 
    {'name' : 'Prithwis'}, 
    {'$set' : { 'age' : 101 }} ) 

list (kollection3.find( {'name' : 'Prithwis' } ))

# Alternative : collection.find_one_and_update( {'name' : Name}, {'$set' : { 'age' : 30 }} ) 

[{'_id': ObjectId('619db1c0c365f4331ba5e274'),
  'address': {'city': 'Kolkata',
   'floor': None,
   'number': 36,
   'postalcode': '700029',
   'street': 'Panditiya Road'},
  'age': 101,
  'favouritefruits': ['apple', 'pineapple', 'orange'],
  'gender': 'Male',
  'likes_code': True,
  'name': 'Prithwis'}]

In [58]:
## Update an existing document
update_result = kollection3.update_one( 
    {'name' : 'Prithwis'}, 
    {'$set' : { 'name' : 'Prithwis Mukerjee' }} ) 

list (kollection3.find( {'name' : 'Prithwis Mukerjee' } ))

[{'_id': ObjectId('619db1c0c365f4331ba5e274'),
  'address': {'city': 'Kolkata',
   'floor': None,
   'number': 36,
   'postalcode': '700029',
   'street': 'Panditiya Road'},
  'age': 101,
  'favouritefruits': ['apple', 'pineapple', 'orange'],
  'gender': 'Male',
  'likes_code': True,
  'name': 'Prithwis Mukerjee'}]

## Additional stuff 
https://www.analyticsvidhya.com/blog/2020/02/mongodb-in-python-tutorial-for-beginners-using-pymongo/

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)